In [2]:
import pandas as pd 
import numpy as np
import geopandas as gpd 
import time
import openpyxl

In [3]:
import os 
os.chdir('/Users/kimmh/Desktop/서데캠/')
os.getcwd()

'/Users/kimmh/Desktop/서데캠'

In [109]:
data = pd.read_excel('./data/all_charger.xlsx', usecols = [0,4,6,7], skiprows=[0,1])
data.columns = ['org', 'area', 'add', 'hour']
data.head()

,org,area,add,hour
0,환경부(한국자동차환경협회),전라북도,전라북도 익산시 동서로 103,24시간 이용가능
1,환경부(한국자동차환경협회),전라남도,전라남도 영암군 신북면 간은정로 21-7,24시간 이용가능
2,환경부(한국자동차환경협회),전라남도,전라남도 영암군 신북면 간은정로 21-7,24시간 이용가능
3,환경부(한국자동차환경협회),강원도,강원도 고성군 거진읍 거진리 22-93,24시간 이용가능
4,환경부(한국자동차환경협회),강원도,강원도 고성군 거진읍 거진리 22-93,24시간 이용가능


In [110]:
data['area'].unique()

array(['전라북도 ', '전라남도 ', '강원도 ', '경상남도 ', '경상북도 ', '광주광역시 ', '서울특별시 ',
       '충청남도 ', '경기도 ', '충청북도 ', '대전광역시 ', '세종특별자치시 ', '대구광역시 ', '부산광역시 ',
       '제주특별자치도 ', '인천광역시 ', '울산광역시 '], dtype=object)

#### 서울만 추출

In [111]:
dt = data[data['area']=='서울특별시 ']
print(dt.shape)
dt = dt.reset_index(drop=True)
dt.head()

(11672, 4)


,org,area,add,hour
0,환경부(한국자동차환경협회),서울특별시,서울특별시 마포구 매봉산로 37,24시간 이용가능
1,환경부(한국자동차환경협회),서울특별시,서울특별시 마포구 매봉산로 80,24시간 이용가능
2,환경부(한국자동차환경협회),서울특별시,서울특별시 마포구 매봉산로 80,24시간 이용가능
3,환경부(한국자동차환경협회),서울특별시,서울특별시 강남구 남부순환로3104,24시간 이용가능
4,환경부(한국자동차환경협회),서울특별시,서울특별시 강남구 개포로 621,24시간 이용가능


area : 서울특별시 우측의 공백 제거 

In [112]:
dt['area'] = dt['area'].apply(lambda x: x.rstrip())

#### 일단 서울만 뽑은 테이블 새 데이터로 저장 

In [113]:
# dt.to_csv('all_charger_seoul.txt', sep=',', index=False)

---

### 서울 충전소 데이터

In [114]:
from itertools import repeat

In [115]:
data = pd.read_table('all_charger_seoul.txt', sep=',')
data['cnt'] = list(repeat(1, len(data)))
print(data.shape)
data.head()

(11672, 5)


,org,area,add,hour,cnt
0,환경부(한국자동차환경협회),서울특별시,서울특별시 마포구 매봉산로 37,24시간 이용가능,1
1,환경부(한국자동차환경협회),서울특별시,서울특별시 마포구 매봉산로 80,24시간 이용가능,1
2,환경부(한국자동차환경협회),서울특별시,서울특별시 마포구 매봉산로 80,24시간 이용가능,1
3,환경부(한국자동차환경협회),서울특별시,서울특별시 강남구 남부순환로3104,24시간 이용가능,1
4,환경부(한국자동차환경협회),서울특별시,서울특별시 강남구 개포로 621,24시간 이용가능,1


이 데이터에선 같은 장소의 충전소 여러대를 일일이 행 구분하여 작성했음  
따라서 개수를 세기 위해 cnt 열 추가

#### hour 결측치 보간

In [116]:
data.isna().sum()

org       0
area      0
add       0
hour    772
cnt       0
dtype: int64

In [118]:
data = data.fillna('정보없음')

In [119]:
data['hour'].isna().sum()

0

In [122]:
dt = data.groupby(['org', 'area', 'add', 'hour'])
gr = dt.sum('cnt')
gr = gr.reset_index()

print(gr.shape)
gr.head()

(3533, 5)


,org,area,add,hour,cnt
0,GS칼텍스,서울특별시,서울특별시 강남구 논현로 508 (GS타워 지하5층),정보없음,2
1,GS칼텍스,서울특별시,서울특별시 강남구 논현로 516주유소 입구 부근(역삼동 668-13),24시간 이용가능,1
2,GS칼텍스,서울특별시,서울특별시 강남구 봉은사로113길 51동호주유소,24시간 이용가능,2
3,GS칼텍스,서울특별시,서울특별시 강남구 삼성로 563주유소 입구 부근,24시간 이용가능,2
4,GS칼텍스,서울특별시,서울특별시 강동구 양재대로 1323신월주유소,24시간 이용가능,2


In [123]:
gr['cnt'].sort_values(ascending=False)

439     57
720     54
1795    46
443     42
102     40
        ..
754      1
752      1
751      1
2787     1
2953     1
Name: cnt, Length: 3533, dtype: int64

In [126]:
over_ten = gr[gr['cnt']>=20]
over_ten.sort_values(by='cnt', ascending=False)

,org,area,add,hour,cnt
439,스타코프,서울특별시,서울특별시 송파구 올림픽로43길 88(388-1 서울아산병원),09:00~18:00,57
720,에버온,서울특별시,"서울특별시 서초구 신반포로 270 (반포동, 반포자이아파트)","24시간 이용가능,입주민만 사용가능 거주자외출입제한",54
1795,차지비,서울특별시,"서울특별시 송파구 올림픽로 300지하4층 N3, M3, N5 구역",24시간 이용가능,46
443,스타코프,서울특별시,서울특별시 송파구 잠실로 88레이크팰리스(44 레이크팰리스),09:00~18:00,42
102,기타,서울특별시,서울특별시 양천구 목동동로 363파리공원(906 파리공원),24시간 이용가능,40
2770,한국전력,서울특별시,서울특별시 강남구 자곡동 자곡로 260,24시간 이용가능,38
333,스타코프,서울특별시,서울특별시 구로구 벚꽃로 370하늘공원 공영주차장(1-11 하늘공원주차장),09:00~18:00,37
2670,한국전기차충전서비스,서울특별시,"서울특별시 영등포구 여의대로 108지하 3층, 4층, 5층 1I, 1J 기둥(22 ...",24시간 이용가능,36
441,스타코프,서울특별시,서울특별시 송파구 위례북로 30위례 A1-5BL(위례포레샤인 17단지),09:00~18:00,33
989,에스트래픽,서울특별시,서울특별시 종로구 우정국로 26(공평동 17),24시간 이용가능,32


주소가 개더럽네...